梯度下降--让模型逼近最小偏差
# 梯度下降的作用及分类
梯度下降法是一个最优算法，常用于机器学习和人工智能中递归性地逼近最小偏差模型，梯度下降的方向也就是用负梯度方向为搜索方向，沿着梯度下降的方向求解极小值。

在训练过程中，每次的正向传播后都会得到输出值与真实值的损失值，这个损失值越小，代表模型越好，于是梯度下降的算法就用在这里，帮助寻找最小的那个损失值，从而可以反推出对应的学习参数b和w，达到优化模型的效果。

> 常用的梯度下降方法可以分为：**批量梯度下降**、**随机梯度下降**和**小批量梯度下降**。

- 批量梯度下降：遍历全部数据集算一次损失函数，然后算函数对各个参数的梯度和更新梯度。这种方法每更新一次参数，都要把数据集里的所有样本看一遍，计算量大，计算速度慢，不支持在线学习，成为Batch gradient descent，批量梯度下降。
- 随机梯度下降：每看一个数据就算一下损失函数，然后求梯度更新参数，这称为stochastic gradient descent，随机梯度下降。这个方法速度比较快，但是收敛性能不太好，可能在最优点附近晃来晃去，命中不到最优点。两次参数的更新也有可能互相抵消，造成目标函数震荡比较剧烈。
- 小批量梯度下降：为了克服上面两种方法的缺点，一般采用一种折中手段--小批量梯度下降。这种方法把数据分为若干个批次，按批来更新参数，这样一批中的一组数据共同决定了本次梯度的方向，下降起来就不容易跑偏，减少了随机性。另一方面因为批的样本数与整个数据集相比小了很多，计算量也不是很大。

# TensorFlow中的梯度下降函数
在TensorFlow中通过一个叫Optimizer的优化器类进行训练优化的。对于不同算法的优化器，在TensorFlow中会有不同的类。
![梯度下降优化器](imgs/13_gradient_descent.png)

在训练过程中，先实例一个优化函数如tf.train.GradientDescentOptimizer,并基于一定的学习率进行梯度优化训练：
___
```Python
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
```
___
接着使用一个minimize()操作，里面传入损失值节点loss，再启动一个外层的循环，优化器就会按照循环的次数一次次沿着loss最小值的方向优化参数。

整个过程中的求导和反向传播操作，都是在优化器里自动完成的。目前比较常用的优化器为**Adam**优化器。

# 退化学习率--在训练的速度和精度之间找到平衡
设置学习率的大小，是在精度和速度之间找到一个平衡：
- 如果学习率的值太大，则训练速度会提升，但结果精度不够；
- 如果学习率的值比较小，精度虽然提升了，但训练会耗费太多时间；

退化学习率又叫学习率衰减，它的本意就是希望在训练过程中对于学习率大和小的优点都能够为我们所用，当训练刚开始时使用大的学习率加速训练，训练到一定程度后使用小的学习率来提高精度，这时可以使用学习率衰减的方法：
___
```Python
def exponential_decay(learning_rate,global_step,decay_steps,decay_rate,staircase=False,name=None):
```
___
学习率的衰减速度是由global_step和decay_steps来决定的。
___
```Python
decayed_learning_rate = learning_rate*decay_rate^(global_step/decay_steps)
```
___

staircase值默认为False。当为True时，将没有衰减功能，只是使用上面的公式初始化一个学习率的值而已。
___
```Python
learning_rate=tf.train.exponential_decay(starter_learning_rate,global_step,100000,0.96)
```
___
这种方式定义的学习率就是退化学习率，它的意思是当前迭代到global_step步，学习率每一步都按照每10万步缩小到0.96%的速度衰退。

> 通过增大批次处理样本的数量也可以起到退化学习率的效果。但是这种方法要求训练时的最小批次要与实际应用中的最小批次一致。一旦满足要求该条件时，建议优先选择增大批次数量的方法，因为这样会省去一些开发量和训练中的计算量。

# 退化学习率的用法举例
## 实例描述
定义一个学习率变量，将其衰减系数设置好，并设置好迭代循环次数，将每次迭代运算的次数与学习率打印出来，观察学习率按照次数退化的现象。

In [4]:
import tensorflow as tf
global_step = tf.Variable(0,trainable=False)
initial_learning_rate = 0.1 #
learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate,global_step=global_step,decay_steps=10,decay_rate=0.9)
opt = tf.train.GradientDescentOptimizer(learning_rate)
add_global = global_step.assign_add(1) #定义一个op,令global_step加1完成计步

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(learning_rate))
    
    for i in range(20):
        g,rate = sess.run([add_global,learning_rate]) #循环20补，打印学习率
        print(g,rate)

0.1
1 0.1
2 0.09895193
3 0.09791484
4 0.09688862
5 0.095873155
6 0.09387404
7 0.092890166
8 0.092890166
9 0.09095325
10 0.09095325
11 0.08905673
12 0.08812335
13 0.087199755
14 0.087199755
15 0.08628584
16 0.0853815
17 0.08360115
18 0.08272495
19 0.08272495
20 0.08185793


> 这是一种常用的训练策略，在训练神经网络时，通常在训练刚开始使用较大的learning_rate，随着训练的进行，会慢慢减小learning_rate。在使用时，一定要把当前迭代次数global_step传进去，否则不会有退化的功能。

# 初始化学习参数
在定义学习参数时可以通过get_variable和Variable两个方式，对于一个网络模型，参数不同的初始化情况，对网络的影响会很大，所以在TensorFlow提供了很多不同特性的初始化函数。get_variable定义如下：
___
```Python
def get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None)
```
___
参数initializer是初始化参数，下图为相关函数：
![初始化函数](imgs/13_initializer_1.png)![初始化函数](imgs/13_initializer_2.png)

在tf.contrib.layers函数中还有个tf.contrib.layers.xavier_initailizer初始化函数，用来在所有层中保持梯度答题相同。尤其在深度神经网络里会经常使用。
> 一般常用的初始化函数为tf.truncated_normal函数，因为该函数有截断功能，可以生成相对比较温和的初始值。